In [1]:
from src import utils
from omegaconf import OmegaConf
from src.train_hyperopt import load_train

cfg = OmegaConf.load("..\config\main.yaml")

data = utils.load_data(cfg)
model = utils.load_model(data, cfg)

In [27]:
from sklearn.model_selection import TimeSeriesSplit, cross_validate, cross_val_predict

ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=7,
    max_train_size=None,
    test_size=None,
)
cv_results = cross_validate(
    model,
    data['X_train'],
    data['y_train'].values.ravel(),
    cv=ts_cv,
    return_estimator=True,
    return_train_score=True,
    scoring=[
        "neg_mean_absolute_error",
        "neg_root_mean_squared_error",
        "neg_mean_absolute_percentage_error",
        "r2",
    ],
    n_jobs=-1,
)

mae = -cv_results["test_neg_mean_absolute_error"].mean().round(2)
rmse = -cv_results["test_neg_root_mean_squared_error"].mean().round(2)
mape = -cv_results["test_neg_mean_absolute_percentage_error"].mean().round(2)
r2 = cv_results["test_r2"].mean().round(2) 

metrics = {"rmse": rmse, "mae": mae, "mape": mape, "r2": r2}

In [28]:
metrics

{'rmse': 0.35, 'mae': 0.28, 'mape': 0.03, 'r2': 0.3}

In [4]:
sorted(cv_results.keys())

['estimator',
 'fit_time',
 'score_time',
 'test_neg_mean_absolute_error',
 'test_neg_mean_absolute_percentage_error',
 'test_neg_root_mean_squared_error',
 'test_r2',
 'train_neg_mean_absolute_error',
 'train_neg_mean_absolute_percentage_error',
 'train_neg_root_mean_squared_error',
 'train_r2']